In [1]:
import csv, json, requests
from datetime import datetime
import pandas as pd
from requests.auth import HTTPBasicAuth
import configparser

In [2]:
# bring in the information about all resources on data.austintexas.gov as of 3 Oct 2016; 
# file source = socrata resource for admins called "Dataset of Datasets", accessed 3 Oct 2016

with open('all_views_20161003.csv', mode='r') as infile:
    reader = csv.DictReader(infile)
    data = []
    for row in reader:
        data.append(row)
# check on the number of records in the file
    print("The number of Socrata resources in this file is " + str(len(data)) + ".")

The number of Socrata resources in this file is 1457.


In [3]:
# an example of what's in a record
data[0].keys()

dict_keys(['Visits', 'Last Update Date (data)', 'Type', 'Creation Date', 'Department', 'Domain', 'Name', 'Downloads', 'U ID', 'Category', 'Owner', 'Keywords', 'ContactEmail', 'Derived View', 'Description', 'Frequency'])

In [4]:
# we want to filter out the resources that aren't tables. And we don't want derived views. 

# let's start looking at our filtering options 
# find out what kind of information is in the first record:
data[0].keys()

dict_keys(['Visits', 'Last Update Date (data)', 'Type', 'Creation Date', 'Department', 'Domain', 'Name', 'Downloads', 'U ID', 'Category', 'Owner', 'Keywords', 'ContactEmail', 'Derived View', 'Description', 'Frequency'])

In [5]:
# that Type item looks like a possibility
# let's use pandas to explore the data... it's a little easier.

# create a dataframe
df = pd.DataFrame(data)

# take a look at the values available in the Type column
df['Type'].unique()

array(['chart', 'map', 'table', 'data_lens', 'calendar', 'filter/grouped',
       'external dataset', 'story', 'blob'], dtype=object)

In [6]:
# let's find out how many tables there are
df[['Type','U ID']].groupby('Type').count()

,U ID
Type,
blob,120
calendar,3
chart,349
data_lens,12
external dataset,54
filter/grouped,94
map,195
story,188
table,442


In [7]:
# are any of those table records a derived view? because we don't want derived views.
df[['Type', 'Derived View']][df['Type'] == 'table'].groupby('Derived View').count()

,Type
Derived View,
false,442


In [9]:
# ok. let's get the id values for the 442 table recors so we can build urls to query their visit history

b = df[['U ID', 'Type']][df['Type'] == 'table']
table_ids = b['U ID'].values
print(len(table_ids))

442


In [10]:
# generate the list of urls

metrics_urls = []
for i in table_ids:
    url = 'https://data.austintexas.gov/api/views/' + i + '/metrics.json?start=1451606400000&end=1475539199999'
    metrics_urls.append(url)
print('created a list of ' + str(len(metrics_urls)) + ' urls')

created a list of 442 urls


In [13]:
# get ready to call the Socrata API. don't store password in the notebook
config = configparser.ConfigParser()
config.read('secrets.txt')
user = config['socrata']['u']
password = config['socrata']['p']

# make the call for each url in the list. store each response in a dictionary
table_metrics_ytd = []
for u in metrics_urls:
    t = {}
    r = requests.get(u, auth=(user, password))
    i = u[39:48]
    l = 'https://data.austintexas.gov/d/' + i
    f = {'fetched_url': u, 'id': i, 'dataset_page_url': l}
    d = r.json()
    f.update(d)
    table_metrics_ytd.append(f)
print('made ' + str(len(table_metrics_ytd)) + ' api calls.')

made 442 api calls.


In [14]:
# load the data we just got into a data frame and check it out
df2 = pd.DataFrame(table_metrics_ytd)
df2.columns

Index(['charts-created', 'charts-created-total', 'charts-deleted',
       'charts-deleted-total', 'code', 'comments-created',
       'comments-created-total', 'dataset_page_url', 'downloads',
       'downloads-total', 'embeds', 'embeds-total', 'error', 'fetched_url',
       'filters-created', 'filters-created-total', 'filters-deleted',
       'filters-deleted-total', 'govstat-metrics-using',
       'govstat-metrics-using-total', 'govstat-total-computes',
       'govstat-total-computes-total', 'id', 'maps-created',
       'maps-created-total', 'maps-deleted', 'maps-deleted-total', 'message',
       'visits', 'visits-total'],
      dtype='object')

In [19]:
# check to see if any url calls returned an error

df2['code'].unique()

array([nan, 'not_found'], dtype=object)

In [20]:
# rut-roh!

df2['message'].unique()

array([nan, 'Cannot find view with id fe74-agg3',
       'Cannot find view with id 8r2c-upwq',
       'Cannot find view with id xqf2-udyn',
       'Cannot find view with id 9r6u-h97f',
       'Cannot find view with id w4x7-gfja',
       'Cannot find view with id 7358-krk7',
       'Cannot find view with id daek-f7x4',
       'Cannot find view with id avj9-39zb',
       'Cannot find view with id jir8-be2g',
       'Cannot find view with id uhvc-ynxd',
       'Cannot find view with id bc5r-88dy',
       'Cannot find view with id 5nzk-szvg',
       'Cannot find view with id ezdt-euam',
       'Cannot find view with id qs9r-eh8x',
       'Cannot find view with id m5q4-3y3d',
       'Cannot find view with id 4kje-y9af',
       'Cannot find view with id p7kc-wkbc',
       'Cannot find view with id tqgd-mf4x',
       'Cannot find view with id ywgb-2ig8',
       'Cannot find view with id qyst-3472',
       'Cannot find view with id 3zr8-huug',
       'Cannot find view with id karz-jr5v',
     

In [21]:
# how many datasets returned an error?
df2[['code', 'id']][df2['code'] == 'not_found'].groupby('code').count()

,id
code,
not_found,80


In [24]:
# make a list of them

df2[['id']][df2['code'] == 'not_found'].values

array([['fe74-agg3'],
       ['8r2c-upwq'],
       ['xqf2-udyn'],
       ['9r6u-h97f'],
       ['w4x7-gfja'],
       ['7358-krk7'],
       ['daek-f7x4'],
       ['avj9-39zb'],
       ['jir8-be2g'],
       ['uhvc-ynxd'],
       ['bc5r-88dy'],
       ['5nzk-szvg'],
       ['ezdt-euam'],
       ['qs9r-eh8x'],
       ['m5q4-3y3d'],
       ['4kje-y9af'],
       ['p7kc-wkbc'],
       ['tqgd-mf4x'],
       ['ywgb-2ig8'],
       ['qyst-3472'],
       ['3zr8-huug'],
       ['karz-jr5v'],
       ['ez4t-k8n4'],
       ['36qt-hi3h'],
       ['fhx8-fyu2'],
       ['yisx-jwus'],
       ['y5qk-ycha'],
       ['uf4q-4zh3'],
       ['put4-j4ni'],
       ['mkfw-xvtp'],
       ['kzja-82u7'],
       ['jady-amz8'],
       ['555q-48w2'],
       ['vimt-emci'],
       ['nywn-y4pr'],
       ['d4m6-5bq8'],
       ['qjby-4sji'],
       ['342j-aqe3'],
       ['ajmb-qjqw'],
       ['e68g-thne'],
       ['p4g8-g35g'],
       ['qz79-qws3'],
       ['wj3f-puun'],
       ['7tg6-n9nk'],
       ['axjn-4unb'],
       ['q

In [18]:
# write the metrics to a csv so more people can explore it

x = table_metrics_ytd
keys = df2.columns
with open('table_metrics_ytd.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(x)